# WCA Database - Automated Download and Import

Created by Michael George (AKA Logiqx)

v1.0 - 2016-03-08

## Database Parameters

Update the connection details for your MySQL environment.

In [2]:
database = "DATABASE"
username = "USERNAME"
password = "PASSWORD"

## Download the HTML

Fetch the database export  page from the WCA website.

In [10]:
# The library urllib2 will be used for the download
import urllib2

# Specify the URL
base_url = "https://www.worldcubeassociation.org/results/misc/"

# Specify the URL
html_url = base_url + "export.html"

# Open the URL and retrieve the TML
page = urllib2.urlopen(html_url)

## Parse the HTML

Locate the link for the ZIP file containing the SQL script.

In [11]:
# The library Beautiful Soup will be used for the HTML parsing
from bs4 import BeautifulSoup

# Parse the HTML using beautiful soup
soup = BeautifulSoup(page, "lxml")

# Find all of the links on the page
links = soup.find_all("a")

# Iterate through links
for link in links:
    
    # Get the hyperlink reference
    href = link.get("href")
    
    # If it is the SQL script it's the one that we want
    if href.endswith(".sql.zip"):
        
        # Record the SQL and exit the loop
        zip_fn = href
        zip_url = base_url + href
        break

## Download the ZIP

Save the ZIP to the local machine.

In [12]:
# Create file handle for the ZIP
infile = urllib2.urlopen(zip_url)

# Write the ZIP to a local file
with open(zip_fn, "wb") as outfile:
    outfile.write(infile.read())

# Close the URL
infile.close()

## Extract the SQL

Extract the SQL script from within the ZIP file.

In [13]:
# Use the zipfile library to handle the zipfile
import zipfile

# Open the ZIP file
zipfile = zipfile.ZipFile(zip_fn, "r")

# Iterate through members
for member in zipfile.namelist():
    
    # Is it the SQL?
    if member.endswith(".sql"):
        
        # Extract the SQL
        zipfile.extract(member)

# Close the ZIP file
zipfile.close()

## Run the SQL

Run the SQL script to re-create the database

Note: The actual database is expected to exist already!

In [14]:
# Use the OS libreary to execute mysql script
import os

# Create the command
cmd = 'mysql --database=%s --user=%s --password=%s --execute="source WCA_export.sql" --default-character-set=utf8' % (database, username, password)

# Execute the command
os.system(cmd)

0

## Create Indices

Create table indices to improve query performance.

In [3]:
# Use the PyMySQL library for adhoc SQL
import pymysql

# Open the connection
conn = pymysql.connect(host='localhost', user=username, passwd=password, db=database)

# Run the SQL commands
c = conn.cursor() 
c.execute("CREATE INDEX person ON persons (id)")
c.execute("CREATE INDEX resultsperson ON results (personId)")
c.execute("CREATE INDEX competitions ON competitions (id)")
c.close() 

# Close the connection
conn.close() 

# All Done!